In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from safetensors.torch import load
from scipy.signal import butter, filtfilt, iirnotch

In [39]:
silent_volts_path = '/content/yaro_silent_4_volts.csv'
silent_volts_df = pd.read_csv(silent_volts_path).iloc[:, :6]

In [ ]:
#plt.plot(silent_volts_chunks[0]);
#print(get_prediction())

In [ ]:
#def get_prediction():
#  pass

In [40]:
silent_volts_df

,CH1,CH2,CH3,CH4,CH5,CH6
0,0.0140,0.0115,0.0219,0.0185,0.00547,0.00611
1,0.0140,0.0115,0.0219,0.0184,0.00547,0.00610
2,0.0140,0.0116,0.0219,0.0184,0.00544,0.00609
3,0.0140,0.0115,0.0219,0.0184,0.00542,0.00607
4,0.0140,0.0116,0.0219,0.0185,0.00546,0.00610
...,...,...,...,...,...,...
40995,0.0141,0.0120,0.0220,0.0181,0.00643,0.00644
40996,0.0141,0.0120,0.0220,0.0181,0.00643,0.00643
40997,0.0141,0.0120,0.0219,0.0181,0.00642,0.00640
40998,0.0141,0.0120,0.0219,0.0181,0.00640,0.00639


In [41]:
def butter_bandpass(lowcut, highcut, fs, order=4):
    nyq = 0.5 * fs
    low = lowcut / nyq
    high = highcut / nyq
    b, a = butter(order, [low, high], btype='band')
    return b, a

def butter_bandpass_filter(data, lowcut, highcut, fs, order=4):
    b, a = butter_bandpass(lowcut, highcut, fs, order=order)
    y = filtfilt(b, a, data)
    return y

def apply_notch_filter(data, notch_freq, fs, quality_factor=30):
    b, a = iirnotch(notch_freq / (0.5 * fs), quality_factor)
    y = filtfilt(b, a, data)
    return y

# Parameters for the filters
fs = 250  # Sampling rate in Hz
lowcut = 1.3  # Low cut frequency in Hz
highcut = 50  # High cut frequency in Hz, adjusted for movement artifact avoidance
notch_freq = 60  # Notch frequency for power line noise in Hz

# Apply the Butterworth bandpass filter
for column in silent_volts_df.columns:
    silent_volts_df[column] = butter_bandpass_filter(silent_volts_df[column].values, lowcut, highcut, fs, order=4)

# Apply the notch filter
for column in silent_volts_df.columns:
    silent_volts_df[column] = apply_notch_filter(silent_volts_df[column].values, notch_freq, fs)




silent_volts_df = silent_volts_df.to_numpy()
silent_volts_df = silent_volts_df[:750, :]

AttributeError: 'numpy.ndarray' object has no attribute 'columns'

In [43]:
silent_volts_df.shape

(41000, 6)

In [3]:
def get_prediction(silent_volts_df):
  def butter_bandpass(lowcut, highcut, fs, order=4):
      nyq = 0.5 * fs
      low = lowcut / nyq
      high = highcut / nyq
      b, a = butter(order, [low, high], btype='band')
      return b, a

  def butter_bandpass_filter(data, lowcut, highcut, fs, order=4):
      b, a = butter_bandpass(lowcut, highcut, fs, order=order)
      y = filtfilt(b, a, data)
      return y

  def apply_notch_filter(data, notch_freq, fs, quality_factor=30):
      b, a = iirnotch(notch_freq / (0.5 * fs), quality_factor)
      y = filtfilt(b, a, data)
      return y

  # Parameters for the filters
  fs = 250  # Sampling rate in Hz
  lowcut = 1.3  # Low cut frequency in Hz
  highcut = 50  # High cut frequency in Hz, adjusted for movement artifact avoidance
  notch_freq = 60  # Notch frequency for power line noise in Hz

  # Apply the Butterworth bandpass filter
  for column in silent_volts_df.columns:
      silent_volts_df[column] = butter_bandpass_filter(silent_volts_df[column].values, lowcut, highcut, fs, order=4)

  # Apply the notch filter
  for column in silent_volts_df.columns:
      silent_volts_df[column] = apply_notch_filter(silent_volts_df[column].values, notch_freq, fs)




  silent_volts_df = silent_volts_df.to_numpy()

  assert silent_volts_df.shape == (6, 750), "Shape of silent_volts_df is not correct"

  ####### ####### ####### ############## ####### ####### #######

  class SimpleFNN(nn.Module):
      def __init__(self, input_size):
          super(SimpleFNN, self).__init__()
          self.fc1 = nn.Linear(input_size, 128)
          self.fc2 = nn.Linear(128, 64)
          self.fc3 = nn.Linear(64, 1)

      def forward(self, x):
          x = F.relu(self.fc1(x))
          x = F.relu(self.fc2(x))
          x = torch.sigmoid(self.fc3(x))
          return x

  # Assuming your numpy array and silent_volts_chunks are defined
  X = np.array([chunk.flatten() for chunk in silent_volts_chunks])
  X_tensor = torch.FloatTensor(X)

  # Assuming 'your_model_safetensors_path' is the path to your saved SafeTensors model file
  your_model_safetensors_path = '/content/SimpleFNN_1229.st'

  # Initialize your model
  device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
  input_size = X.shape[1]  # Number of features
  model = SimpleFNN(input_size)

  # Before calling load, open the file in binary mode and read its content
  with open(your_model_safetensors_path, 'rb') as f:
      model_state_bytes = f.read()

  # Now pass the bytes object to the load function
  model_state = load(model_state_bytes)

  # Directly load the state dict into your model
  model.load_state_dict(model_state)

  model.to(device)

  # Ensure the model is in evaluation mode
  model.eval()

  # Get predictions for the first input
  with torch.no_grad():
      inputs = X_tensor[:1].to(device)  # Assuming you want to predict the first input
      outputs = model(inputs)
      predicted = outputs.round()  # Assuming binary classification


torch.Size([1, 4500])

In [37]:
predicted

tensor([[0.]])